# Dithering Example

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.interpolate import RegularGridInterpolator

import  astro3d
from astro3d import image_stack

plt.style.use([{'image.cmap':'gray_r'}])

load data, but we take just a small subset

In [ ]:
f = np.load(astro3d.get_data('turbulentbox.npy'))
_data = f.copy()
del f

data = _data[:, :, 0]
data = data[data.shape[0]//2:, data.shape[1]//2:]

In [ ]:
n_highres = 400

define a norm

In [ ]:
vmax = _data.max()
norm = LogNorm(1e-3 * vmax, vmax, clip=True)

d_00 = np.array(norm(data))

define an interpolation function

In [ ]:
f_interp = RegularGridInterpolator(list(np.arange(i) for i in list(data.shape) + [1]), data[:, :, None])

coords = [
    np.linspace(0, data.shape[0] - 1, n_highres),
    np.linspace(0, data.shape[1] - 1, n_highres),
    [0]
    ]

Define color mapping

In [ ]:
colors = [image_stack.BaseCyan, image_stack.BaseMagenta]
levels = [0.2, 0.7]
sigmas = [0.1, 0.15]
clips = 3 * np.ones(len(levels))

In [ ]:
image_stack.show_histogram(data, norm, colors=colors, levels=levels, sigmas=sigmas, clips=clips)

Compute color-dithering

In [ ]:
ld, im = astro3d.image_stack.makeslice(0, [0], f_interp, coords, norm, None, levels=levels, sigmas=sigmas, colors=colors, clip=1, fill=0.8)

Plot displaying the different steps

In [ ]:
from scipy.interpolate import interp2d

f, ax = plt.subplots(2, 2, figsize=(10, 10), gridspec_kw={'hspace':0.12, 'wspace':0.12})

_x = np.linspace(0, d_00.shape[0], n_highres)
_y = np.linspace(0, d_00.shape[1], n_highres)

# use color-dithered image
d_01 = im.transpose(1, 0, 2)

# interpolate original data
d_10 = interp2d(np.arange(d_00.shape[0]), np.arange(d_00.shape[1]), d_00, kind='cubic')(_x, _y)

# dither interpolated data
d_11 = astro3d.fmodule.dither(d_10)

wx = 0.5
wy = 0.5
dw = 0.15

d = [d_00, d_01, d_10, d_11]

for _ax, _d in zip(ax.ravel(), d):
    
    _ax.imshow(_d, vmin=0, vmax=1, origin='lower')

    _axins = _ax.inset_axes([0.05, 0.05, 0.3, 0.3])
    _axins.imshow(_d, vmin=0, vmax=1, origin="lower")
    _axins.set_xlim(wx * _d.shape[0], (wx + dw) * _d.shape[0])
    _axins.set_ylim(wy * _d.shape[1], (wy + dw) * _d.shape[1])
    _ax.indicate_inset_zoom(_axins, edgecolor='k')
    
    for __ax in [_ax, _axins]:
        __ax.set_xticks([])
        __ax.set_yticks([])

def tprops(ax):
    return dict(va='top', ha='right', fontsize='x-large', transform=ax.transAxes, bbox=dict(boxstyle='round', facecolor='0.85', alpha=0.75))

ax[0,0].text(0.95, 0.95, 'original data', **tprops(ax[0, 0]))
ax[1,0].text(0.95, 0.95, 'interpolated data', **tprops(ax[1, 0]))
ax[0,1].text(0.95, 0.95, 'color-dithered', **tprops(ax[0, 1]))
ax[1,1].text(0.95, 0.95, 'dithered', **tprops(ax[1, 1]))


prop1 = dict(fontsize='40', va='center')
prop2 = dict(fontsize='10', va='center')

pos = ax[0, 0].get_position()
f.text(pos.x1 + 0.05 * pos.width, pos.y0 - 0.08 * pos.height, r'$\mathsf{\Rightarrow}$', **prop1, rotation=45, ha='center')
f.text(pos.x1 + 0.05 * pos.width + 0.04 * pos.height, pos.y0 - 0.12 * pos.height, r'color-dither', **prop2, rotation=45, ha='center')

pos = ax[1,0].get_position()
f.text(pos.x1 + 0.05 * pos.width, pos.y0 + 0.5 * pos.height, '$\mathsf{\Rightarrow}$', **prop1, ha='center')
f.text(pos.x1 + 0.06 * pos.width, pos.y0 + 0.4 * pos.height, 'dither', **prop2, ha='center')

f.text(pos.x0 + 0.5 * pos.width, pos.y1 + 0.05 * pos.height, '$\mathsf{\Downarrow}$', **prop1, ha='center')
f.text(pos.x0 + 0.57 * pos.width, pos.y1 + 0.06 * pos.height, 'interpolate', **prop2, ha='left')

    
f.savefig('example.pdf', transparent=True, bbox_inches='tight', dpi=300)